# 计算图

In [1]:
import tensorflow as tf

In [2]:
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = a+b

In [4]:
#tf.get_default_graph()获取当前默认的计算图
#可以通过a.graph查看张量所属的计算图。因为没有指定所以这个计算图应该等于当前默认的计算图。
print(a.graph is tf.get_default_graph())

True


In [6]:
#tf.Graph()来生成新的计算图，不同计算图上的张量和运算都不会共享
g1 = tf.Graph()
with g1.as_default():
    #在计算图g1中定义变量‘v’，并设置初始值为0.
    v = tf.get_variable("v",shape=[1],initializer = tf.zeros_initializer)
    
g2 = tf.Graph()
with g2.as_default():
    #在计算图g1中定义变量‘v’，并设置初始值为1.
    v = tf.get_variable("v",shape=[1],initializer = tf.ones_initializer)
    
with tf.Session(graph = g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse=True):
        #在计算图g1中，变量‘v’的取值应该为0，所以下面会输出[0.]
        print(sess.run(tf.get_variable("v")))
with tf.Session(graph = g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse=True):
        #在计算图g2中，变量‘v’的取值应该为1，所以下面会输出[0=1.]
        print(sess.run(tf.get_variable("v")))

    

[ 0.]
[ 1.]


In [7]:
#tf.Graph.device()用来指定运行计算的设备
g = tf.Graph()
#指定计算运行的装备
with g.device('/gpu:0'):
    result = a+b

# 张量

In [8]:
#tf.constant()是一个计算，这个计算结果为一个张量，保存在变量a
a = tf.constant([1.0,2.0],name = "a")
b = tf.constant([2.0,3.0],name = "b")
result = tf.add(a,b,name = "add")
print(result)

Tensor("add_2:0", shape=(2,), dtype=float32)


TersorFlow计算结果不是一个具体的数字，而是一个张量的结构。
- 张量结构保存了三个属性：名字；维度；类型。
- 其中张量的命名是通过"node:src_output"的形式给出。node是节点的名称，src_output表示当前张量来自节点的第几个输出。
- shape给出计算结果的维度
- type表示类型，参与运算的张量必须类型匹配，否则会报错

In [11]:
#类型不匹配报错
#a = tf.constant([1,2],name="a")
a = tf.constant([1,2],name = "a",dtype=tf.float32)#可以指定类型
b = tf.constant([2.0,3.0],name = "b")
result = a+b

## 张量的使用

In [14]:
#使用张量计算中间结果
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = a+b
print(result)
#直接计算向量的和
result = tf.constant([1.0,2.0],name="a")+\
tf.constant([2.0,3.0],name="b")
print(result)

Tensor("add_7:0", shape=(2,), dtype=float32)
Tensor("add_8:0", shape=(2,), dtype=float32)


# TensorFlow 运行模型——会话
## 创建会话的两种方式,所有的计算完成后都必须关闭会话。

In [24]:
#创建一个会话
sess = tf.Session()
#使用这个创建好的会话得到关心的运算的结果。
print(sess.run(result))
#关闭会话使得本次运行的资源被释放
sess.close()

[ 3.  5.]


In [25]:
# 创建一个会话，并通过Python的上下文管理器来管理这个会话
with tf.Session() as sess:
    print(sess.run(result))
#不需要调用“Session.close()”函数来关闭会话，当上下文退出会话,关闭和资源释放就自动完成了


[ 3.  5.]


In [19]:
#TensorFlow不会自动生成默认的会话，需要手动指定。
#可以通过tf.Tensor.eval函数计算一个张量的取值
sess = tf.Session()
with sess.as_default():
    print(result.eval())

[ 3.  5.]


In [26]:
#这个代码也可以完成相同的功能。
sess = tf.Session()
#以下两条代码功能一样
print(sess.run(result))
print(result.eval(session= sess))
sess.close()

[ 3.  5.]
[ 3.  5.]


In [27]:
#交互式环境下，可以通过tf.InteractiveSession()函数设置默认会话
sess = tf.InteractiveSession()
print(result.eval())
sess.close()

[ 3.  5.]


## 通ConfigProtocol Protocol Buffer配置会话


In [28]:
config = tf.ConfigProto(allow_soft_placement=True,\
                       log_device_placement=True)
sess1 = tf.InteractiveSession(config = config)
sess2 = tf.Session(config = config)


- allow_soft_placement:当它为True，满足某些条件，GPU运算可以放在CPU上进行。默认为False,一般设置为True.
- log_device_placement：当它为True，日志中将会记录每个节点被安排在哪个设备以方便调试。False:可以减少日志量。